<a href="https://colab.research.google.com/github/ZwL6/MZT/blob/main/assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark
!pip install -U -q PyDrive
!apt install openjdk-8-jdk-headless -qq
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

openjdk-8-jdk-headless is already the newest version (8u372-ga~us1-0ubuntu1~20.04).
0 upgraded, 0 newly installed, 0 to remove and 13 not upgraded.


In [2]:
import pyspark
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

In [3]:
# Get cities file
!wget https://raw.githubusercontent.com/ZwL6/MZT/main/cities.json



--2023-06-28 18:13:27--  https://raw.githubusercontent.com/ZwL6/MZT/main/cities.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15829 (15K) [text/plain]
Saving to: ‘cities.json.1’

cities.json.1       100%[===================>]  15.46K  --.-KB/s    in 0s      

2023-06-28 18:13:27 (56.7 MB/s) - ‘cities.json.1’ saved [15829/15829]



In [4]:
#making standerd route
import json
import random

# Set seed
random.seed(1)
with open('cities.json', 'r') as file:
    data = json.load(file)

cities = [d['city'] for d in data]

merchandise = ['apple', 'tomatoes', 'butter', 'water', 'milk', 'pens', 'honey']

routes = []

for i in range(1000):
    route = {}
    route['id'] = i

    # Generate a list of non-repeating random cities
    city_sample = random.sample(cities, 3)

    # Randomly generate the number of merchandise
    num_merchandise = random.randint(1, 10)

    # Randomly select merchandise types
    merchandise_sample = random.sample(merchandise, random.randint(3, 5))

    # Build the route dictionary format
    route['route'] = []
    for j in range(2):
        route_info = {}
        route_info['from'] = city_sample[j]
        route_info['to'] = city_sample[j+1]

        # Generate random merchandise quantities for each route
        merchandise_dict = {}
        for m in merchandise_sample:
            merchandise_dict[m] = random.randint(1, num_merchandise)

        route_info['merchandise'] = merchandise_dict

        route['route'].append(route_info)

    routes.append(route)

# Write the route list to a JSON file without corrupt records
valid_routes = [r for r in routes if 'route' in r and len(r['route']) == 2]
with open('routes.json', 'w') as file:
    json.dump(routes, file, indent=4)


In [5]:
# Download routes
# from google.colab import files
# files.download('routes.json')


In [6]:
# making actual route

import json
import random
import copy

# Set seed
random.seed(2)

# Read the JSON file
with open('cities.json', 'r') as file:
    data = json.load(file)

# Extract city names
cities = [item['city'] for item in data]

# Define merchandise types
merchandise_types = ['apple', 'tomatoes', 'butter', 'water', 'milk', 'pens', 'honey']

with open('routes.json', 'r') as file:
    routes = json.load(file)

act_routes = []

for route in routes:
    if len(route['route']) < 2:
        continue  # Skip routes with less than two cities

    for _ in range(10):
        act_route = copy.deepcopy(route)  # Use deepcopy to avoid modifying the original route

        # Randomly select a city to modify
        modified_city_index = random.randint(0, len(act_route['route']) - 1)
        modified_city = act_route['route'][modified_city_index]

        # Randomly decide whether to modify city
        if random.choice([True, False]):
            # Modify the selected city
            new_city = random.choice(cities)
            modified_city['from'] = new_city

            # If the modified city is the second city, modify it throughout the route
            if modified_city_index == 1:
                # Replace the modified city with a new city
                act_route['route'][modified_city_index]['from'] = new_city
                # Connect the cities before and after the replaced city
                act_route['route'][modified_city_index - 1]['to'] = new_city
                if modified_city_index + 1 < len(act_route['route']):
                    act_route['route'][modified_city_index]['to'] = act_route['route'][modified_city_index + 1]['from']

        # Randomly modify merchandise
        for city_route in act_route['route']:
            # Select random merchandise types
            selected_merchandise = random.sample(merchandise_types, random.randint(1, len(merchandise_types)))
            # Assign random quantities to selected merchandise
            city_route['merchandise'] = {item: random.randint(1, 5) for item in selected_merchandise}

        act_routes.append(act_route)

# Write the act routes list to a new JSON file
with open('act_routes.json', 'w') as file:
    json.dump(act_routes, file, indent=4)

In [7]:
# Download the actual routes
# from google.colab import files
# files.download('act_routes.json')

In [8]:
!pip install python-Levenshtein
import json
from collections import defaultdict
from Levenshtein import distance

# Function for calculating route edit distance similarity
def calculate_similarity(route1, route2):
    lev_distance = distance(route1, route2)
    max_length = max(len(route1), len(route2))
    similarity = 1 - lev_distance / max_length
    return similarity

# Function for calculating marchandise edit distance similarity
def calculate_merchandise_similarity(merch1, merch2):
    lev_distance = distance(merch1, merch2)
    max_length = max(len(merch1), len(merch2))
    similarity = 1 - lev_distance / max_length
    return similarity

# open standard route file
with open('routes.json', 'r') as file:
    routes = json.load(file)

# open actual route file
with open('act_routes.json', 'r') as file:
    act_routes = json.load(file)

# Create a dictionary to store the most similar route for each actual route
most_similar_routes = defaultdict(list)

# Correct prediction counter
correct_predictions = 0

# Iterate through all actual routes
for act_route in act_routes:
    act_route_id = act_route['id']
    act_route_str = ' '.join([f"{route['from']} {route['to']}" for route in act_route['route']])

    similarities = []

    # route similarity calculating
    for standard_route in routes:
        standard_route_id = standard_route['id']
        standard_route_str = ' '.join([f"{route['from']} {route['to']}" for route in standard_route['route']])

        route_similarity = calculate_similarity(standard_route_str, act_route_str)

        # merchandise similarity calculating
        merchandise_similarity = 0
        for i, route in enumerate(act_route['route']):
            act_merchandise = route['merchandise']
            standard_merchandise = standard_route['route'][0]['merchandise']
            merchandise_similarity += calculate_merchandise_similarity(str(standard_merchandise), str(act_merchandise))
        merchandise_similarity /= len(standard_route['route'])

        #total similarity calculating
        total_similarity = (route_similarity + merchandise_similarity) / 2
        similarities.append((standard_route_id, total_similarity))

    # sort the max similarity
    similarities = sorted(similarities, key=lambda x: x[1], reverse=True)

    # Find the most similar route
    max_similarity = similarities[0]

    # Store the most similar route for this actual route
    most_similar_routes[act_route_id].append(max_similarity)

    # If the actual route id is the same as the predicted route id, increment the correct prediction counter
    if act_route_id == max_similarity[0]:
        correct_predictions += 1

# Print the most similar routes for all actual routes
# for act_route_id, max_similarities in most_similar_routes.items():
#    for i, max_similarity in enumerate(max_similarities):
#        print(f"Max similarity for actual route {act_route_id}-{i+1}: Standard Route {max_similarity[0]} with similarity {max_similarity[1]}")

# Calculate and print the prediction accuracy
accuracy = correct_predictions / len(act_routes)
print(f"Prediction accuracy: {accuracy * 100}%")



Prediction accuracy: 82.38%


In [20]:
# Convert 'standard routes' file to Spark DataFrame
import pandas as pd
import json
import warnings
warnings.filterwarnings("ignore")

# Load data from JSON file
with open('routes.json', 'r') as file:
    data = json.load(file)

# Initialize an empty DataFrame to store the processed data
df = pd.DataFrame()

# Iterate over each element
for element in data:
  id = element['id']

# Iterate over routes
  for route in element['route']:
    from_city = route['from']
    to_city = route['to']

    # Iterate over merchandise
    for merchandise, quantity in route['merchandise'].items():
        # Append data to DataFrame
        df = df.append({
            'id': id,
            'from': from_city,
            'to': to_city,
            'merchandise': merchandise,
            'quantity': quantity
        }, ignore_index=True)

# Convert DataFrame to Spark DataFrame
st_routes_df = spark.createDataFrame(df)

In [35]:
from pyspark.sql.functions import expr

# Create a new column that is the merchandise column value repeated quantity times
str_df = st_routes_df.withColumn('merchandise_quantity', expr("repeat(merchandise, quantity)"))
str_df = str_df.drop('merchandise','quantity')

str_df.show()

+---+------------+------------+--------------------+
| id|        from|          to|merchandise_quantity|
+---+------------+------------+--------------------+
|  0|     Helmond| Schoonhoven|waterwaterwaterwater|
|  0|     Helmond| Schoonhoven|          honeyhoney|
|  0|     Helmond| Schoonhoven|                pens|
|  0| Schoonhoven|     Zaandam|waterwaterwaterwater|
|  0| Schoonhoven|     Zaandam|               honey|
|  0| Schoonhoven|     Zaandam|    penspenspenspens|
|  1|      Nuenen|Klazienaveen|penspenspenspensp...|
|  1|      Nuenen|Klazienaveen|            tomatoes|
|  1|      Nuenen|Klazienaveen|                milk|
|  1|      Nuenen|Klazienaveen|               apple|
|  1|Klazienaveen|   The Hague|                pens|
|  1|Klazienaveen|   The Hague|tomatoestomatoest...|
|  1|Klazienaveen|   The Hague|    milkmilkmilkmilk|
|  1|Klazienaveen|   The Hague|appleappleappleap...|
|  2|    Enschede|   Harlingen|    milkmilkmilkmilk|
|  2|    Enschede|   Harlingen|tomatoestomatoe

In [70]:
# delete the duplicated routes
stpaths = str_df.dropDuplicates(["id", "from", "to"])

stpaths.show()

+---+-------------+------------+--------------------+
| id|         from|          to|merchandise_quantity|
+---+-------------+------------+--------------------+
|  0|      Helmond| Schoonhoven|waterwaterwaterwater|
|  0|  Schoonhoven|     Zaandam|waterwaterwaterwater|
|  1| Klazienaveen|   The Hague|                pens|
|  1|       Nuenen|Klazienaveen|penspenspenspensp...|
|  2|     Enschede|   Harlingen|    milkmilkmilkmilk|
|  2|    Harlingen|      Venray|milkmilkmilkmilkm...|
|  3|     De Meern|  Heerenveen|honeyhoneyhoneyho...|
|  3|   Heerenveen|  Schaesberg|honeyhoneyhoneyho...|
|  4|  Bloemendaal|     Rucphen|            milkmilk|
|  4|      Rucphen|    Oostzaan|                milk|
|  5|   Amersfoort|        Echt|                pens|
|  5|        Ommen|  Amersfoort|                pens|
|  6|       Dongen|        Leek|    milkmilkmilkmilk|
|  6|     Reeuwijk|      Dongen|milkmilkmilkmilkmilk|
|  7|    Rozenburg| Zwijndrecht|               honey|
|  7|  Zwijndrecht|   Schijn

In [30]:
import pandas as pd
import json
from pyspark.sql import SparkSession
from pyspark.sql.functions import monotonically_increasing_id

# Load data from JSON file
with open('act_routes.json', 'r') as file:
    data = json.load(file)

# Initialize an empty DataFrame to store the processed data
df = pd.DataFrame()

# Dictionary to store the counter for each unique ID
id_counter = {}

# Iterate over each element
for element in data:
    id = element['id']

    # Increment the counter for each unique ID
    if id in id_counter:
        id_counter[id] += 1
    else:
        id_counter[id] = 1

    # Counter for each ID
    counter = id_counter[id]

    # Iterate over routes
    for route in element['route']:
        from_city = route['from']
        to_city = route['to']

        # Iterate over merchandise
        for merchandise, quantity in route['merchandise'].items():
            # Append data to DataFrame with modified ID
            df = df.append({
                'id': f"{id}-{counter}",  # Assign modified ID with counter
                'original_id': id,  # Store original ID
                'from': from_city,
                'to': to_city,
                'merchandise': merchandise,
                'quantity': quantity
            }, ignore_index=True)

# Convert DataFrame to Spark DataFrame
spark = SparkSession.builder.getOrCreate()
ac_routes_df = spark.createDataFrame(df)

# Assign sequential counts to duplicate IDs
ac_routes_df = ac_routes_df.withColumn('new_id', monotonically_increasing_id())


+---+-----------+-----------+-----------+-----------+--------+------+
| id|original_id|       from|         to|merchandise|quantity|new_id|
+---+-----------+-----------+-----------+-----------+--------+------+
|0-1|          0|Barendrecht|Schoonhoven|      honey|       3|     0|
|0-1|          0|Barendrecht|Schoonhoven|   tomatoes|       5|     1|
|0-1|          0|Barendrecht|Schoonhoven|     butter|       2|     2|
|0-1|          0|Schoonhoven|    Zaandam|      apple|       4|     3|
|0-1|          0|Schoonhoven|    Zaandam|       milk|       5|     4|
|0-1|          0|Schoonhoven|    Zaandam|   tomatoes|       3|     5|
|0-1|          0|Schoonhoven|    Zaandam|      water|       5|     6|
|0-1|          0|Schoonhoven|    Zaandam|     butter|       4|     7|
|0-2|          0|    Helmond|   Enschede|      water|       4|     8|
|0-2|          0|    Helmond|   Enschede|     butter|       5|     9|
|0-2|          0|    Helmond|   Enschede|      honey|       2|    10|
|0-2|          0|   

In [44]:
acr_df = ac_routes_df.withColumn('merchandise_quantity', expr("repeat(merchandise, quantity)"))
acr_df = acr_df.drop('merchandise', 'quantity', 'new_id')

acr_df.show()


+---+-----------+-----------+-----------+--------------------+
| id|original_id|       from|         to|merchandise_quantity|
+---+-----------+-----------+-----------+--------------------+
|0-1|          0|Barendrecht|Schoonhoven|     honeyhoneyhoney|
|0-1|          0|Barendrecht|Schoonhoven|tomatoestomatoest...|
|0-1|          0|Barendrecht|Schoonhoven|        butterbutter|
|0-1|          0|Schoonhoven|    Zaandam|appleappleappleapple|
|0-1|          0|Schoonhoven|    Zaandam|milkmilkmilkmilkmilk|
|0-1|          0|Schoonhoven|    Zaandam|tomatoestomatoest...|
|0-1|          0|Schoonhoven|    Zaandam|waterwaterwaterwa...|
|0-1|          0|Schoonhoven|    Zaandam|butterbutterbutte...|
|0-2|          0|    Helmond|   Enschede|waterwaterwaterwater|
|0-2|          0|    Helmond|   Enschede|butterbutterbutte...|
|0-2|          0|    Helmond|   Enschede|          honeyhoney|
|0-2|          0|   Enschede|    Zaandam|tomatoestomatoest...|
|0-2|          0|   Enschede|    Zaandam|          hone

In [66]:
# delete the duplicated routes
acpaths = acr_df.dropDuplicates(["id", "original_id", "from", "to"])

acpaths.show()

+----+-----------+-----------+-----------+--------------------+
|  id|original_id|       from|         to|merchandise_quantity|
+----+-----------+-----------+-----------+--------------------+
| 0-1|          0|Barendrecht|Schoonhoven|     honeyhoneyhoney|
| 0-1|          0|Schoonhoven|    Zaandam|appleappleappleapple|
|0-10|          0| Den Helder|Schoonhoven|          waterwater|
|0-10|          0|Schoonhoven|    Zaandam|    penspenspenspens|
| 0-2|          0|   Enschede|    Zaandam|tomatoestomatoest...|
| 0-2|          0|    Helmond|   Enschede|waterwaterwaterwater|
| 0-3|          0|    Helmond|   Nootdorp|waterwaterwaterwater|
| 0-3|          0|   Nootdorp|    Zaandam|     waterwaterwater|
| 0-4|          0|    Helmond|Schoonhoven|penspenspenspenspens|
| 0-4|          0|Schoonhoven|    Zaandam|honeyhoneyhoneyho...|
| 0-5|          0|    Helmond|Schoonhoven|tomatoestomatoest...|
| 0-5|          0|Schoonhoven|    Zaandam|appleappleappleap...|
| 0-6|          0| Amstelveen|    Zaanda

In [50]:
from pyspark.sql.functions import udf
from pyspark.sql.types import IntegerType
from Levenshtein import distance

def levenshtein_distance(s1, s2):
    return distance(s1, s2)

levenshtein_distance_udf = udf(levenshtein_distance, IntegerType())
